# Unit testing

Source code is (most typically) divided into classes and functions.

*Unit testing* consists in testing individual units of code (e.g., a single function) to ensure that each behaves as intended.
Note that unit tests define an _expected behaviour_, they do not enforce a specific implementation. Therefore, unit tests can be defined before any line of source code is actually written!

## Some remarks

[General]
* Unit tests can detect unexpected behaviours, but they cannot tell you where you made a mistake (that's still for you to figure out)
* **Unit tests also enforce a syntax**. This is very useful to avoid breaking changes later in your project.
* **Unit tests can evolve too**: you can (and sometimes, should) modify your tests to reflect changes in the code.
* Some useful links:
    * [Unit testing in Julia](https://docs.julialang.org/en/stable/stdlib/Test/)
    * [Unit testing in Python](https://docs.python.org/3/library/unittest.html)

[After you've run the example]
* **You can't catch what you don't test**: if we had written the test with `a=2.0`, the test would have passed the second time too. In general, you may want to test (within reason) all scenarios defined in the source code. Here, we may 

# A simple example: weighted sum of two vectors

Suppose we code a function `weighted_sum` (see code below) that computes the weighted sum of two vectors.

Symbolically, given two vectors $X, Y$ and scalars $a, b$, we want to compute $Z$ as:
$$
    Z := aX + bY
$$

## Unit test for `weighted_sum`

Because the behaviour of that function is fairly simple, we can write a unit test for it right away.

In [1]:
using LinearAlgebra
using Test

In [2]:
"""
    test_ws()

Perform a simple test of the `weighted_sum` function.
"""
function test_ws()
    a = 1.0
    b = 2.0
    X = [1.0, 1.0]
    Y = [4.0, 5.0]

    # Run the function on that test example
    Z = weighted_sum(a, X, b, Y)

    # Check that Z takes the right values
    @test Z == [9.0, 11.0]
end

test_ws

If we run the test now, it will raise an error since the function `weighted_sum` has not been defined yet.

In [3]:
# At this point, this should raise an error
test_ws()

UndefVarError: UndefVarError: weighted_sum not defined

## A first implementation

Let's begin by writing a simple implementation of `weighted_sum`.

Note that the unit test above not only verifies a certain behaviour, it also defines a given syntax for calling the function: the first argument is $a$, the second argument is $X$, etc...

When implementing `weighted_sum`, we have to follow this syntax, otherwise the above test might fail.

In [4]:
"""
    weighted_sum(a, X, b, Y)

Compute the weighted sum `a*X + b*Y` and return the result.
"""
function weighted_sum(a::Float64, X::Vector{Float64}, b::Float64, Y::Vector{Float64})
    
    # Sanity check (ensure both vectors have same length)
    n = length(X)
    n == length(Y) || throw(DimensionMismatch(
        "X has size $n but Y has size $(length(Y))"
    ))
    
    # Creates a zero-valued vector of length `n`
    # Z will hold the result
    Z = zeros(Float64, n)  
    
    # Compute weighted sum
    for i in 1:n
        Z[i] = a * X[i] + b * Y[i]
    end
    
    return Z
end

weighted_sum

In [5]:
# Execute the test
test_ws()

Test Passed

## Modifying the source code

Later in the project, we notice that if $a=1$, then there is no need to compute the product $a\times X_{i}$, since this simply equals $X_{i}$. Let's modify `weighted_sum` accordingly.

In [6]:
"""
    weighted_sum!(a, X, b, Y)

Overwrite `X` with `a*X + b*Y`. 
"""
function weighted_sum(a::Float64, X::Vector{Float64}, b::Float64, Y::Vector{Float64})
    
    # Sanity check
    n = length(X)
    n == length(Y) || throw(DimensionMismatch(
        "X has size $n but Y has size $(length(Y))"
    ))
    
    Z = zeros(Float64, n)
    
    # Compute weighted sum
    if a == 1.0
        for i in 1:n
            Z[i] = X[i] - b * Y[i]
        end
        
    else
        for i in 1:n
            Z[i] = a * X[i] + b * Y[i]
        end
    end
    
    return Z
end

weighted_sum

Notice the (intentional) mistake on line `19`:
```julia
    Z[i] = X[i] - b * Y[i]  # /!\ should have been X[i] + b * Y[i] /!\
```
Although this is clearly a bug, it will not raise an error at runtime: the function will run smoothly, the result will simply be wrong.

Also, note that this bug arises only when $a=1$. The default loop has no error.

In [7]:
# Although there is an error in the code, the function still runs smoothly
# However, it returns a wrong result
Z = weighted_sum(1.0, [1.0, 1.0], 2.0, [1.0, 1.0])

2-element Array{Float64,1}:
 -1.0
 -1.0

In [8]:
# However, this time the test will fail
test_ws()

Test Failed at In[2]:16
  Expression: Z == [9.0, 11.0]
   Evaluated: [-7.0, -9.0] == [9.0, 11.0]


Test.FallbackTestSetException: There was an error during testing